---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [63]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# # you can use the following function to plot graphs
# # make sure to comment it out before submitting to the autograder
# def plot_graph(G, weight_name=None):
#     '''
#     G: a networkx G
#     weight_name: name of the attribute for plotting edge weights (if G is weighted)
#     '''
#     %matplotlib notebook
#     import matplotlib.pyplot as plt
    
#     plt.figure()
#     pos = nx.spring_layout(G)
#     edges = G.edges()
#     weights = None
    
#     if weight_name:
#         weights = [int(G[u][v][weight_name]) for u,v in edges]
#         labels = nx.get_edge_attributes(G,weight_name)
#         nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
#         nx.draw_networkx(G, pos, edges=edges, width=weights);
#     else:
#         nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [1]:
def answer_one():
    g_df = pd.read_csv('Employee_Movie_Choices.txt',sep='\t',skiprows=(1),names=['Employee','Movie'])
    graph = nx.from_pandas_dataframe(g_df,'Employee','Movie')
    return graph

In [4]:
# !cat 'Employee_Movie_Choices.txt'

#Employee	Movie
Andy	Anaconda
Andy	Mean Girls
Andy	The Matrix
Claude	Anaconda
Claude	Monty Python and the Holy Grail
Claude	Snakes on a Plane
Frida	The Matrix
Frida	The Shawshank Redemption
Frida	The Social Network
Georgia	Anaconda
Georgia	Monty Python and the Holy Grail
Georgia	Snakes on a Plane
Joan	Forrest Gump
Joan	Kung Fu Panda
Joan	Mean Girls
Lee	Forrest Gump
Lee	Kung Fu Panda
Lee	Mean Girls
Pablo	The Dark Knight
Pablo	The Matrix
Pablo	The Shawshank Redemption
Vincent	The Godfather
Vincent	The Shawshank Redemption
Vincent	The Social Network


### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [25]:
def answer_two():
    g_df = pd.read_csv('Employee_Movie_Choices.txt',sep='\t',skiprows=(1),names=['Employee','Movie'])
    movies = g_df['Movie'].values
    employees = g_df['Employee'].values
    
    graph = nx.from_pandas_dataframe(g_df,'Employee','Movie')
    
    for node in graph.nodes():
        if node in movies:
            graph.add_node(node,type='movie')
        else:
            graph.add_node(node,type='employee')
            
    return graph
    

In [29]:
graph = answer_two()
graph.nodes(data=True)

[('Andy', {'type': 'employee'}),
 ('Anaconda', {'type': 'movie'}),
 ('Mean Girls', {'type': 'movie'}),
 ('The Matrix', {'type': 'movie'}),
 ('Claude', {'type': 'employee'}),
 ('Monty Python and the Holy Grail', {'type': 'movie'}),
 ('Snakes on a Plane', {'type': 'movie'}),
 ('Frida', {'type': 'employee'}),
 ('The Shawshank Redemption', {'type': 'movie'}),
 ('The Social Network', {'type': 'movie'}),
 ('Georgia', {'type': 'employee'}),
 ('Joan', {'type': 'employee'}),
 ('Forrest Gump', {'type': 'movie'}),
 ('Kung Fu Panda', {'type': 'movie'}),
 ('Lee', {'type': 'employee'}),
 ('Pablo', {'type': 'employee'}),
 ('The Dark Knight', {'type': 'movie'}),
 ('Vincent', {'type': 'employee'}),
 ('The Godfather', {'type': 'movie'})]

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [30]:
def answer_three():
    graph = answer_two()
    g_df = pd.read_csv('Employee_Movie_Choices.txt',sep='\t',skiprows=(1),names=['Employee','Movie'])
    employees = set(g_df['Employee'].values)
    P = bipartite.weighted_projected_graph(graph,employees)
    
    return P

In [33]:
graph = answer_three()
graph.edges(data=True)

[('Vincent', 'Pablo', {'weight': 1}),
 ('Vincent', 'Frida', {'weight': 2}),
 ('Andy', 'Georgia', {'weight': 1}),
 ('Andy', 'Pablo', {'weight': 1}),
 ('Andy', 'Joan', {'weight': 1}),
 ('Andy', 'Lee', {'weight': 1}),
 ('Andy', 'Claude', {'weight': 1}),
 ('Andy', 'Frida', {'weight': 1}),
 ('Georgia', 'Claude', {'weight': 3}),
 ('Pablo', 'Frida', {'weight': 2}),
 ('Joan', 'Lee', {'weight': 3})]

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [61]:
def answer_four():
    graph = answer_three()
    common_movies_df = pd.DataFrame(graph.edges(data=True),columns=['Employee1','Employee2','Movies'])
    common_movies_df['Movies'] = common_movies_df['Movies'].map(lambda x:int(x['weight']))
    
    duplicate = common_movies_df.copy()
    duplicate = duplicate.rename(columns={'Employee1':'Employee2','Employee2':'Employee1'})
    
    common_movies_df = pd.concat([common_movies_df,duplicate],ignore_index=True)
    
    relationships_df = pd.read_csv('Employee_Relationships.txt',sep='\t',names=['Employee1','Employee2','Score'])
    relationships_df['Score'] = relationships_df['Score'].astype(int)
    
    merged = pd.merge(relationships_df,common_movies_df,how='left')
    
    merged['Movies'] = merged['Movies'].fillna(0)
    
    correlation_score = merged['Score'].corr(merged['Movies'])
    
    return correlation_score

In [62]:
answer_four()

0.78839622217334748

In [50]:
relationships_df = pd.read_csv('Employee_Relationships.txt',sep='\t',names=['Employee1','Employee2','Score'])
relationships_df['Score'] = relationships_df['Score'].astype(int)
relationships_df



,Employee1,Employee2,Score
0,Andy,Claude,0
1,Andy,Frida,20
2,Andy,Georgia,-10
3,Andy,Joan,30
4,Andy,Lee,-10
5,Andy,Pablo,-10
6,Andy,Vincent,20
7,Claude,Frida,0
8,Claude,Georgia,90
9,Claude,Joan,0


In [44]:
graph = answer_three()
common_movies_df = pd.DataFrame(graph.edges(data=True),columns=['Employee1','Employee2','Movies'])
common_movies_df['Movies'] = common_movies_df['Movies'].map(lambda x:x['weight'])
common_movies_df


,Employee1,Employee2,Movies
0,Vincent,Pablo,1
1,Vincent,Frida,2
2,Andy,Georgia,1
3,Andy,Pablo,1
4,Andy,Joan,1
5,Andy,Lee,1
6,Andy,Claude,1
7,Andy,Frida,1
8,Georgia,Claude,3
9,Pablo,Frida,2


In [45]:
duplicate = common_movies_df.copy()
duplicate = duplicate.rename(columns={'Employee1':'Employee2','Employee2':'Employee1'})
duplicate



,Employee2,Employee1,Movies
0,Vincent,Pablo,1
1,Vincent,Frida,2
2,Andy,Georgia,1
3,Andy,Pablo,1
4,Andy,Joan,1
5,Andy,Lee,1
6,Andy,Claude,1
7,Andy,Frida,1
8,Georgia,Claude,3
9,Pablo,Frida,2


In [46]:
common_movies_df = pd.concat([common_movies_df,duplicate],ignore_index=True)
common_movies_df

,Employee1,Employee2,Movies
0,Vincent,Pablo,1
1,Vincent,Frida,2
2,Andy,Georgia,1
3,Andy,Pablo,1
4,Andy,Joan,1
5,Andy,Lee,1
6,Andy,Claude,1
7,Andy,Frida,1
8,Georgia,Claude,3
9,Pablo,Frida,2


In [47]:
pd.merge(relationships_df,common_movies_df,how='left')

,Employee1,Employee2,Score,Movies
0,Andy,Claude,0,1.0
1,Andy,Frida,20,1.0
2,Andy,Georgia,-10,1.0
3,Andy,Joan,30,1.0
4,Andy,Lee,-10,1.0
5,Andy,Pablo,-10,1.0
6,Andy,Vincent,20,NaN
7,Claude,Frida,0,NaN
8,Claude,Georgia,90,3.0
9,Claude,Joan,0,NaN


In [34]:
# !cat 'Employee_Relationships.txt'

Andy	Claude	0
Andy	Frida	20
Andy	Georgia	-10
Andy	Joan	30
Andy	Lee	-10
Andy	Pablo	-10
Andy	Vincent	20
Claude	Frida	0
Claude	Georgia	90
Claude	Joan	0
Claude	Lee	0
Claude	Pablo	10
Claude	Vincent	0
Frida	Georgia	0
Frida	Joan	0
Frida	Lee	0
Frida	Pablo	50
Frida	Vincent	60
Georgia	Joan	0
Georgia	Lee	10
Georgia	Pablo	0
Georgia	Vincent	0
Joan	Lee	70
Joan	Pablo	0
Joan	Vincent	10
Lee	Pablo	0
Lee	Vincent	0
Pablo	Vincent	-20
